In [19]:
import os

# takes care of annoying TF-GPU warnings
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# remove useless Tensorflow warning:
# WARNING:absl:Found untraced functions such as _update_step_xla, lstm_cell_1_layer_call_fn, 
# lstm_cell_1_layer_call_and_return_conditional_losses, lstm_cell_2_layer_call_fn, 
# lstm_cell_2_layer_call_and_return_conditional_losses while saving (showing 5 of 5). 
# These functions will not be directly callable after loading.
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [20]:
# very useful for managing wandb runs: https://stackoverflow.com/questions/71106179/log-two-model-runs-with-keras-wandb
import wandb
from wandb.keras import WandbCallback
os.environ["WANDB_SILENT"] = "true"

In [21]:
import multi_classifier_utils as mc_u

#### Logistic Regression: Formula Label Prediction (multi-label, all features)

In [22]:
import numpy as np 
import pandas as pd
from pathlib import Path 
import ast

import tensorflow as tf
import keras_tuner as kt

import tensorflow_datasets as tfds
import tensorflow_text as tf_text

import datetime

tfds.disable_progress_bar()
wandb_project_name = "multi_label_formula_classification"

In [23]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history["val_"+metric], "")
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, "val_"+metric])

#### Import Data and Preprocess Dataframe

In [24]:
def preprocess_data(corpus,
                    irrelevant_features=["mtype",]):
    # drop irrelevant columns
    corpus.drop(irrelevant_features, inplace=True, axis=1)

    def cell_str_to_list(cell_val):
        return ast.literal_eval(cell_val)

    # filter strings
    def process_cell(cell_str):
        stripped_f_str = cell_str[1:-1].replace("\\\\", "\\")
        f_list = stripped_f_str.split(",")
        f_list = [token.replace("'", "").replace(" ", "") for token in f_list]
        f_list = ["{" if token == "\\{" else token for token in f_list]
        f_list = ["}" if token == "\\}" else token for token in f_list]
        cell_str = " ".join(f_list)
        return cell_str

    corpus["type_tokens"] = corpus["type_tokens"].map(process_cell)
    corpus["tokens"] = corpus["tokens"].map(process_cell)
    corpus["mtype_one_hot"] = corpus["mtype_one_hot"].map(cell_str_to_list)
    corpus["labels"] = corpus["labels"].map(cell_str_to_list)
    corpus = corpus.loc[(corpus["tokens"].str.len() > 0) & (corpus["tokens"] != " ")]


In [25]:
#print(os.getcwd())
data_p = Path("../data/") / "multi_class_unbalanced_data_TOKENIZED_V1.csv"
data = pd.read_csv(data_p)
preprocess_data(data)
data.head()

,tokens,type_tokens,labels,labels_str,mtype_one_hot
0,f,func_name __ANON_1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
1,j : \mathbb{N} \rightarrow [ 0 1 ],func_def func_name __ANON_1 COLON mapping set_...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
2,b,func_name __ANON_1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[0, 0, 1]"
3,mathbb{Q,set_constant SET_BASIC,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",['elementary-set-theory'],"[1, 0, 0]"
4,f,func_name __ANON_1,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","['functions', 'elementary-set-theory']","[0, 0, 1]"


In [26]:
print(data["type_tokens"].map(lambda x: len((x.split(" ")))).max())
print(data["tokens"].map(lambda x: len((x.split(" ")))).max())

254
252


In [27]:
# ordinary datasets
SMALL_TRAIN_SIZE = 24620 - 2460
SMALL_TEST_SIZE = 2460
LARGE_TRAIN_SIZE = 106523 - 10650
LARGE_TEST_SIZE = 10650
# compact datasets
NUM_CLASSES = 40

labels_array = np.array(data["labels"].to_list())
m_type_array = np.array(data["mtype_one_hot"].to_list())

dataset4_all_features = tf.data.Dataset.from_tensor_slices((data["tokens"],data["type_tokens"],m_type_array), name="data")
dataset3_tokens_types = tf.data.Dataset.from_tensor_slices((data["tokens"],data["type_tokens"]), name="data")
dataset2_types = tf.data.Dataset.from_tensor_slices((data["type_tokens"]), name="data")
dataset1_tokens = tf.data.Dataset.from_tensor_slices((data["tokens"]), name="data")

labels_ds = tf.data.Dataset.from_tensor_slices(labels_array, name="label")
#data_as_ds = tf.data.Dataset.zip((dat_as_ds, labels_ds))

dataset1_tokens_l = tf.data.Dataset.zip((dataset1_tokens, labels_ds))
dataset2_types_l = tf.data.Dataset.zip((dataset2_types, labels_ds))
dataset3_tokens_types_l = tf.data.Dataset.zip((dataset3_tokens_types, labels_ds))
dataset4_all_features_l = tf.data.Dataset.zip((dataset4_all_features, labels_ds))

In [28]:
test_dataset1 = dataset1_tokens_l.take(SMALL_TEST_SIZE)
train_dataset1 = dataset1_tokens_l.skip(SMALL_TEST_SIZE)
test_dataset2 = dataset2_types_l.take(SMALL_TEST_SIZE)
train_dataset2 = dataset2_types_l.skip(SMALL_TEST_SIZE)
test_dataset3 = dataset3_tokens_types_l.take(SMALL_TEST_SIZE)
train_dataset3 = dataset3_tokens_types_l.skip(SMALL_TEST_SIZE)
test_dataset4 = dataset4_all_features_l.take(SMALL_TEST_SIZE)
train_dataset4 = dataset4_all_features_l.skip(SMALL_TEST_SIZE)

##### Setup and Data Preparation

In [29]:
for (example_token, example_type, example_m_type), label in train_dataset4.take(5):
    print("text: ", example_token.numpy())
    print("type: ", example_type.numpy())
    print("m_type: ", example_m_type.numpy())
    print("label: ", label.numpy())

text:  b'{ 2   4 }'
type:  b'explset L_BRACE_LITERAL set_enumeration item __ANON_3 COMMA item __ANON_3 R_BRACE_LITERAL'
m_type:  [1 0 0]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f : I \\to X'
type:  b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f ( a ) = 1'
type:  b'func_expr func_name_arg func_name __ANON_1 L_PAREN __ANON_1 R_PAREN EQUAL expr_atom __ANON_3'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'x'
type:  b'func_name __ANON_1'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
text:  b'f : X \\to Y'
type:  b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0'
m_type:  [0 0 1]
label:  [1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [30]:
BUFFER_SIZE = 2000
BATCH_SIZE = 64
STEPS_PER_EPOCH = np.floor(SMALL_TRAIN_SIZE/BATCH_SIZE)
VAL_STEPS_PER_EPOCH = np.floor(SMALL_TEST_SIZE/BATCH_SIZE)
#train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
#test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [31]:
test_dataset1 = test_dataset1.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset1 = train_dataset1.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset2 = test_dataset2.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset2 = train_dataset2.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset3 = test_dataset3.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset3 = train_dataset3.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
test_dataset4 = test_dataset4.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)
train_dataset4 = train_dataset4.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

##### Text Encoding

In [32]:
# for int encoder
TYPE_TOKENS_MAX_SEQ_LEN = 260
TOKENS_MAX_SEQ_LEN = 260


# for other encoders 
TYPE_TOKENS_PAD_TO_MAX_TOKENS = 80
TOKENS_PAD_TO_MAX_TOKENS = 200
BIGRAM_PAD_TO_MAX_TOKENS = 350

In [33]:
def adapt_encoder(encoder, mode, dataset, dataset_type):
    if mode == "token": 

        if dataset_type == 1:
            encoder.adapt(dataset.map(lambda inputs, label: inputs))
        elif dataset_type == 2:
            ...
        elif dataset_type == 3:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[0])) # removes the label column through transformation: text, label -> text
        elif dataset_type == 4:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[0])) # removes the label column through transformation: text, label -> text  
    elif mode == "type":
        
        if dataset_type == 1:
            ...
        elif dataset_type == 2:
            encoder.adapt(dataset.map(lambda inputs, label: inputs))
        elif dataset_type == 3:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[1])) # removes the label column through transformation: text, label -> text
        elif dataset_type == 4:
            encoder.adapt(dataset.map(lambda inputs, label: inputs[1])) # removes the label column through transformation: text, label -> text

    return encoder

In [34]:
def create_encoder(output_mode_str, n_grams, mode, dataset, dataset_type):
    if output_mode_str == "int":
        VOCAB_SIZE = 200
        if mode == "token":
            max_seq_len = TOKENS_MAX_SEQ_LEN
        elif mode == "type":
            max_seq_len = TYPE_TOKENS_MAX_SEQ_LEN

        encoder = tf.keras.layers.TextVectorization(
            standardize=None,
            output_mode=output_mode_str,
            ngrams = n_grams,
            output_sequence_length = max_seq_len,
            split="whitespace",
            max_tokens=VOCAB_SIZE)
        #TODO: adapt for different inputs
        encoder = adapt_encoder(encoder, mode, dataset, dataset_type)
        return encoder
    
    if output_mode_str == "count" and n_grams == 2:
        max_seq_len = BIGRAM_PAD_TO_MAX_TOKENS
        encoder = tf.keras.layers.TextVectorization(
            standardize=None,
            output_mode=output_mode_str,
            ngrams = n_grams,
            pad_to_max_tokens = max_seq_len,
            split="whitespace",
            max_tokens=max_seq_len)
        
        
        encoder = adapt_encoder(encoder, mode, dataset, dataset_type)
        return encoder
    
    if mode == "token":
        max_seq_len = TOKENS_PAD_TO_MAX_TOKENS
    elif mode == "type":
        max_seq_len = TYPE_TOKENS_PAD_TO_MAX_TOKENS

    encoder = tf.keras.layers.TextVectorization(
        standardize=None,
        output_mode=output_mode_str,
        ngrams = n_grams,
        pad_to_max_tokens = max_seq_len,
        split="whitespace",
        max_tokens=max_seq_len)
    #TODO: adapt for different inputs
    encoder = adapt_encoder(encoder, mode, dataset, dataset_type)
    return encoder


    

##### Representation 1: Use integer indices encoding

In [35]:
encoder_int_tokens = create_encoder("int", None, "token", train_dataset1, 1)
encoder_int_types = create_encoder("int", None, "type", train_dataset2, 2)

vocab_tokens = np.array(encoder_int_tokens.get_vocabulary())
vocab_size_tokens = len(encoder_int_tokens.get_vocabulary())
vocab_types = np.array(encoder_int_types.get_vocabulary())
vocab_size_types = len(encoder_int_types.get_vocabulary())

print("tokens (voc size): ", vocab_size_tokens)
print("types (voc size): ", vocab_size_types)

tokens (voc size):  128
types (voc size):  61


In [36]:
encoded_example_token = encoder_int_tokens(example_token).numpy()
encoded_example_types = encoder_int_types(example_type).numpy()

print("tokens: ")
print(example_token)
print(encoded_example_token)
print(encoded_example_token.shape)

print("types: ")
print(example_type)
print(encoded_example_types)
print(encoded_example_types.shape)

tokens: 
tf.Tensor(b'f : X \\to Y', shape=(), dtype=string)
[ 2 11 26 14 35  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
(260,)
types: 
tf.Tensor(b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0', shape=(), dtype=string)
[15  5  3 13 14  6 16  6  0  0  

##### Representation 2: Count Vectorizer

In [37]:
encoder_count_tokens = create_encoder("count", None, "token", train_dataset1, 1)
encoder_count_types = create_encoder("count", None, "type", train_dataset2, 2)

vocab_tokens = np.array(encoder_count_tokens.get_vocabulary())
vocab_size_tokens = len(encoder_count_tokens.get_vocabulary())
vocab_types = np.array(encoder_count_types.get_vocabulary())
vocab_size_types = len(encoder_count_types.get_vocabulary())

print("tokens (voc size): ", vocab_size_tokens)
print("types (voc size): ", vocab_size_types)

tokens (voc size):  127
types (voc size):  60


In [38]:
encoded_example_token = encoder_count_tokens(example_token).numpy()
encoded_example_types = encoder_count_types(example_type).numpy()

print("tokens: ")
print(example_token)
print(encoded_example_token)
print(encoded_example_token.shape)

print("types: ")
print(example_type)
print(encoded_example_types)
print(encoded_example_types.shape)

tokens: 
tf.Tensor(b'f : X \\to Y', shape=(), dtype=string)
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
(200,)
types: 
tf.Tensor(b'func_def func_name __ANON_1 COLON mapping __ANON_0 TO __ANON_0', shape=(), dtype=string)
[0. 0. 1. 0. 1. 2. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

##### Model (Logistic Regression)

 **Train the model**

#### Experiment 1: Use integer indices for encoding tokens

##### Model 1: ONLY TOKENS
Find best hyperparameters

In [39]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_tokens1 = create_encoder("int", None, "token", train_dataset1, 1)
model_builder1 = mc_u.create_model_builder1_LR(NUM_CLASSES, encoder_int_tokens1, tokens_input_len)

In [40]:
tuner = kt.Hyperband(model_builder1,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model1_lr',
                     project_name='model1_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [41]:
tuner.search(train_dataset1,
             epochs=30,
             validation_data=test_dataset1,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

Trial 30 Complete [00h 00m 44s]
val_accuracy: 0.2331414520740509

Best val_accuracy So Far: 0.8725329041481018
Total elapsed time: 00h 12m 53s
INFO:tensorflow:Oracle triggered exit


In [42]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

0.0001
96
0.2
0.3


Train with best hyperparameters

In [43]:
NUM_EPOCHS = 80
model1 = mc_u.create_model1_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_int_tokens1, tokens_input_len)
model1.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [44]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_only_tokens",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model1.fit(train_dataset1, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset1,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

Epoch 1/80
344/346 [============================>.] - ETA: 0s - loss: 0.6459 - accuracy: 0.0849 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


346/346 [==============================] - 5s 15ms/step - loss: 0.6456 - accuracy: 0.0861 - recall_1: 0.0000e+00 - val_loss: 0.6061 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 2/80
341/346 [============================>.] - ETA: 0s - loss: 0.5073 - accuracy: 0.2422 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


346/346 [==============================] - 5s 14ms/step - loss: 0.5061 - accuracy: 0.2436 - recall_1: 0.0000e+00 - val_loss: 0.4540 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00
Epoch 3/80
344/346 [============================>.] - ETA: 0s - loss: 0.3460 - accuracy: 0.2890 - recall_1: 0.0000e+00INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


...

346/346 [==============================] - 5s 13ms/step - loss: 0.1337 - accuracy: 0.5886 - recall_1: 0.1308 - val_loss: 0.1210 - val_accuracy: 0.7738 - val_recall_1: 0.2196
Epoch 79/80
343/346 [============================>.] - ETA: 0s - loss: 0.1332 - accuracy: 0.5915 - recall_1: 0.1328INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


346/346 [==============================] - 5s 15ms/step - loss: 0.1335 - accuracy: 0.5890 - recall_1: 0.1322 - val_loss: 0.1210 - val_accuracy: 0.7738 - val_recall_1: 0.2196
Epoch 80/80
346/346 [==============================] - ETA: 0s - loss: 0.1335 - accuracy: 0.5890 - recall_1: 0.1314INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


INFO:tensorflow:Assets written to: /home/paul_d/Sources/sem_math_repo/classification_formulas_multilabel/wandb/run-20230315_233129-0o2wxjlb/files/model-best/assets


346/346 [==============================] - 5s 14ms/step - loss: 0.1335 - accuracy: 0.5890 - recall_1: 0.1314 - val_loss: 0.1209 - val_accuracy: 0.7743 - val_recall_1: 0.2196


##### Model 2: ONLY TYPES
Find best hyperparameters

In [45]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_types2 = create_encoder("int", None, "type", train_dataset2, 2)    
model_builder2 = mc_u.create_model_builder2_LR(NUM_CLASSES, encoder_int_types2,type_input_len)

In [46]:
tuner = kt.Hyperband(model_builder2,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model2_lr',
                     project_name='model2_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)


In [48]:
tuner.search(train_dataset2,
             epochs=30,
             validation_data=test_dataset2,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])


Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
128               |?                 |emb_dims
0.2               |?                 |dropout1
0.3               |?                 |dropout2
0.0001            |?                 |learning_rate
3                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
2                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/3


Traceback (most recent call last):
  File "/home/paul_d/Sources/sem_math_repo/sem_math_env/lib/python3.10/site-packages/keras_tuner/engine/base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/paul_d/Sources/sem_math_repo/sem_math_env/lib/python3.10/site-packages/keras_tuner/engine/base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/paul_d/Sources/sem_math_repo/sem_math_env/lib/python3.10/site-packages/keras_tuner/tuners/hyperband.py", line 419, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/paul_d/Sources/sem_math_repo/sem_math_env/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/paul_d/Sources/sem_math_repo/sem_math_env/lib/python3.10/site-packages/keras_tuner/engine/tuner.py"

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "/home/paul_d/Sources/sem_math_repo/sem_math_env/lib/python3.10/site-packages/keras_tuner/engine/base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/paul_d/Sources/sem_math_repo/sem_math_env/lib/python3.10/site-packages/keras_tuner/engine/base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/paul_d/Sources/sem_math_repo/sem_math_env/lib/python3.10/site-packages/keras_tuner/tuners/hyperband.py", line 419, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/paul_d/Sources/sem_math_repo/sem_math_env/lib/python3.10/site-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/paul_d/Sources/sem_math_repo

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

Train with best hyperparameters

In [ ]:
NUM_EPOCHS = 80
model2 = mc_u.create_model2_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_int_types2, type_input_len)
model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [ ]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_only_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model2.fit(train_dataset2, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset2,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

##### Model 3: TOKENS AND TYPES
Find best hyperparameters

In [ ]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_tokens3 = create_encoder("int", None, "token", train_dataset3, 3)
encoder_int_types3 = create_encoder("int", None, "type", train_dataset3, 3)
model_builder3 = mc_u.create_model_builder3_LR(NUM_CLASSES, encoder_int_tokens3,encoder_int_types3, tokens_input_len, type_input_len)

In [ ]:
tuner = kt.Hyperband(model_builder3,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model3_lr',
                     project_name='model3_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
tuner.search(train_dataset3,
             epochs=30,
             validation_data=test_dataset3,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

Train with best hyperparameters

In [ ]:
NUM_EPOCHS = 80
model3 = mc_u.create_model3_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_int_tokens3, encoder_int_types3, tokens_input_len, type_input_len)
model3.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [ ]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_tokens_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model3.fit(train_dataset3, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset3,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

##### Model 4: TOKENS AND TYPES AND SEM MATH LABELS
Find best hyperparameters

In [ ]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_int_tokens4 = create_encoder("int", None, "token", train_dataset4, 4)
encoder_int_types4 = create_encoder("int", None, "type", train_dataset4, 4)
model_builder4 = mc_u.create_model_builder4_LR(NUM_CLASSES, encoder_int_tokens4,encoder_int_types4, tokens_input_len, type_input_len, "int")

In [ ]:
tuner = kt.Hyperband(model_builder4,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model4_lr',
                     project_name='model4_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
tuner.search(train_dataset4,
             epochs=30,
             validation_data=test_dataset4,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

Train with best hyperparameters

In [ ]:
NUM_EPOCHS = 80
model4 = mc_u.create_model4_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_int_tokens4, encoder_int_types4, tokens_input_len, type_input_len, "int")
model4.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [ ]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_all_features",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "int",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model4.fit(train_dataset4, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset4,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

#### Experiment 2: Use count vectorizer

##### Model 1: ONLY TOKENS
Find best hyperparameters

In [ ]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_tokens1 = create_encoder("count", None, "token", train_dataset1, 1)
model_builder1c = mc_u.create_model_builder1_LR(NUM_CLASSES, encoder_count_tokens1,tokens_input_len)

In [ ]:
tuner = kt.Hyperband(model_builder1c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model1c_lr',
                     project_name='model1c_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
tuner.search(train_dataset1,
             epochs=30,
             validation_data=test_dataset1,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

Train with best hyperparameters

In [ ]:
NUM_EPOCHS = 80
model1c = mc_u.create_model1_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_count_tokens, tokens_input_len)
model1c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [ ]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_only_tokens",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model1c.fit(train_dataset1, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset1,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

##### Model 2: ONLY TYPES
Find best hyperparameters

In [ ]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_types = create_encoder("count", None, "type", train_dataset2, 2)
model_builder2c = mc_u.create_model_builder2_LR(NUM_CLASSES, encoder_count_types, type_input_len)

In [ ]:
tuner = kt.Hyperband(model_builder2c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model2c_lr',
                     project_name='model2c_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
tuner.search(train_dataset2,
             epochs=30,
             validation_data=test_dataset2,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

Train with best hyperparameters

In [ ]:
NUM_EPOCHS = 80
model2c = mc_u.create_model2_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_count_types, type_input_len)
model2c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [ ]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_only_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model2c.fit(train_dataset2, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset2,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

##### Model 3: TOKENS AND TYPES
Find best hyperparameters

In [ ]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_tokens3 = create_encoder("count", None, "token", train_dataset3, 3)
encoder_count_types3 = create_encoder("count", None, "type", train_dataset3, 3)
model_builder3c = mc_u.create_model_builder3_LR(NUM_CLASSES, encoder_count_tokens3,encoder_count_types3, tokens_input_len, type_input_len)

In [ ]:
tuner = kt.Hyperband(model_builder3c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model3c_lr',
                     project_name='model3c_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
tuner.search(train_dataset3,
             epochs=30,
             validation_data=test_dataset3,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

Train with best hyperparameters

In [ ]:
NUM_EPOCHS = 80
model3c = mc_u.create_model3_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_count_tokens3, encoder_count_types3, tokens_input_len, type_input_len)
model3c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [ ]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_tokens_types",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model3c.fit(train_dataset3, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset3,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()

##### Model 4: TOKENS, TYPES, SEM MATH LABELS
Find best hyperparameters

In [ ]:
tokens_input_len = TOKENS_MAX_SEQ_LEN 
type_input_len = TYPE_TOKENS_MAX_SEQ_LEN
NUM_CLASSES = 40

encoder_count_tokens4 = create_encoder("count", None, "token", train_dataset4, 4)
encoder_count_types4 = create_encoder("count", None, "type", train_dataset4, 4)
model_builder4c = mc_u.create_model_builder4_LR(NUM_CLASSES, encoder_count_tokens4, encoder_count_types4, tokens_input_len, type_input_len, "float")

In [ ]:
tuner = kt.Hyperband(model_builder4c,
                     objective=kt.Objective("val_accuracy", direction="max"),
                     max_epochs=20,
                     factor=3,
                     directory='meta_dir/model4c_lr',
                     project_name='model4c_lr')
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
tuner.search(train_dataset4,
             epochs=30,
             validation_data=test_dataset4,
             #steps_per_epoch = STEPS_PER_EPOCH,
             validation_steps = VAL_STEPS_PER_EPOCH,
             callbacks= [stop_early])#[tensorboard_callback])

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
optimal_lr = best_hps.get("learning_rate")
optimal_emb_dims = best_hps.get("emb_dims")
dp1 = best_hps.get("dropout1")
dp2 = best_hps.get("dropout2")
print(optimal_lr)
print(optimal_emb_dims)
print(dp1)
print(dp2)

Train with best hyperparameters

In [ ]:
NUM_EPOCHS = 80
model4c = mc_u.create_model4_LR(NUM_CLASSES, optimal_emb_dims, dp1, dp2, encoder_count_tokens4, encoder_count_types4, tokens_input_len, type_input_len, "float")
model4c.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(optimal_lr),
              metrics=["accuracy", tf.keras.metrics.Recall()])

In [ ]:
configs_dict = {
    "learning_rate": optimal_lr,
    "emb_dim": optimal_emb_dims,
    "dp1": dp1, 
    "dp2": dp2,
    "algorithm": "LogReg",
    "configuration": "multi_all_features",
    "optimizer": "adam",
    "loss": "binary_crossentropy",
    "epochs": NUM_EPOCHS,
    "batch_size": 64,
    "vectorizer": "count",
    "dataset": "multi_class_unbalanced_data_TOKENIZED_V1"
}

run = wandb.init(project=wandb_project_name, reinit=True, config=configs_dict)

history = model4c.fit(train_dataset4, 
                    epochs=NUM_EPOCHS,
                    validation_data=test_dataset4,
                    #steps_per_epoch = STEPS_PER_EPOCH,
                    validation_steps = VAL_STEPS_PER_EPOCH,
                    callbacks= [WandbCallback()])#[tensorboard_callback])
run.finish()